In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('satellite\data\merged_kayrros_mars_carbonmapper_11122025_coal.csv')
df = df.sort_values(by=["country","date"])

df

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\rebek\AppData\Local\Temp\ipykernel_32100\336742702.py:1: SyntaxWarning: invalid escape sequence '\d'
  df = pd.read_csv('satellite\data\merged_kayrros_mars_carbonmapper_11122025_coal.csv')


,sensor,country,sector,date,emission rate (kg/h),uncertainty (kg/h),latitude,longitude,comments,platform
0,Tanager,Australia,Coal,01/03/2025,1182.621727,120.515437,-21.659691,147.968904,Downloaded from https://data.carbonmapper.org/...,Carbon Mapper
1,Tanager,Australia,Coal,01/03/2025,3891.047334,218.557848,-21.887993,147.994376,Downloaded from https://data.carbonmapper.org/...,Carbon Mapper
2,Sentinel-5P/TROPOMI - ESA,Australia,Coal,01/05/2020,31330.000000,14100.000000,-22.841200,148.657400,Downloaded from https://methanewatch.kayrros.c...,Kayrros Methane Watch
3,Tanager,Australia,Coal,01/05/2025,3266.028734,247.312109,-34.181489,150.720350,Downloaded from https://data.carbonmapper.org/...,Carbon Mapper
4,Tanager,Australia,Coal,01/05/2025,1138.798341,373.978657,-34.206934,150.770962,Downloaded from https://data.carbonmapper.org/...,Carbon Mapper
...,...,...,...,...,...,...,...,...,...,...
4960,Sentinel-5P/TROPOMI - ESA,Zimbabwe,Coal,03/06/2025,21000.000000,9000.000000,-18.490000,26.370000,Downloaded from https://methanedata.unep.org/m...,IMEO MARS
4961,Sentinel-5P/TROPOMI - ESA,Zimbabwe,Coal,04/07/2025,18000.000000,6000.000000,-18.500000,26.390000,Downloaded from https://methanedata.unep.org/m...,IMEO MARS
4962,Sentinel-5P/TROPOMI - ESA,Zimbabwe,Coal,10/08/2025,20000.000000,12000.000000,-18.360000,26.570000,Downloaded from https://methanedata.unep.org/m...,IMEO MARS
4963,Sentinel-5P/TROPOMI - ESA,Zimbabwe,Coal,19/06/2025,5000.000000,4000.000000,-18.420000,26.600000,Downloaded from https://methanedata.unep.org/m...,IMEO MARS


In [3]:
# Step 1: Identify duplicates based on sensor, date, lat, lon
dup_keys = ["sensor", "date", "latitude", "longitude"]

# Step 2: Keep only those groups where platform differs
df['platform_count'] = df.groupby(dup_keys)['platform'].transform('nunique')

# Step 3: Filter only rows with >1 unique platform
df_dupes = df[df['platform_count'] > 1].copy()

# Step 4: Define a helper to pick the best record from each group
def pick_best(group):
    # Prefer rows with both emission rate and uncertainty not null
    has_both = group[
        group['emission rate (kg/h)'].notna() &
        group['uncertainty (kg/h)'].notna()
    ]
    if not has_both.empty:
        return has_both.iloc[0]  # keep the first of those
    else:
        return group.iloc[0]     # otherwise keep the first in the group

# Step 5: Apply per-group selection
result_dupes = df_dupes.groupby(dup_keys, group_keys=False).apply(pick_best)

# Step 6: Combine with the rest of the data (non-duplicates)
result = pd.concat([
    df[df['platform_count'] == 1],  # unique platform
    result_dupes
], ignore_index=True)

# Step 7: Drop the helper column
result = result.drop(columns='platform_count')

result = result.sort_values(by=["country","date"])

C:\Users\rebek\AppData\Local\Temp\ipykernel_32100\1187132962.py:23: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_dupes = df_dupes.groupby(dup_keys, group_keys=False).apply(pick_best)


In [4]:
result.to_csv('final_data.csv', index=False)